In [9]:
import json
import networkx as nx

In [6]:
a = ["911", "DOMESTICTERRORWEB", "suicide", "MAIL", "HEROIN_DEALING", "HAMBURG_TIE_YEAR", "SWINGERS_club"]
data = {i: j for i, j in enumerate(a)}

In [22]:
G = read_sample("911")
nodes = G_nodes(G)
links = G_links(G)
centrality = node_centrality_criteria(G)

In [20]:
with open("links.json", "w") as file:
    json.dump(links, file)

with open("nodes.json", "w") as file:
    json.dump(links, file)


In [25]:
ranking = {
    "degree_ranking": { n: v for n, v in zip(centrality["degree"]["node"], centrality["degree"]["value"])},
    "closeness_ranking": { n: v for n, v in zip(centrality["closeness"]["node"], centrality["closeness"]["value"])},
    "betweenness_ranking": { n: v for n, v in zip(centrality["betweenness"]["node"], centrality["betweenness"]["value"])},
    "page_rank_ranking": { n: v for n, v in zip(centrality["page_rank"]["node"], centrality["page_rank"]["value"])},
}

with open("ranking.json", "w") as file:
    json.dump(ranking, file)


In [16]:
def G_nodes(G, graph_layout={}):

    degree = {node: degree for (node, degree) in G.degree()}
    closeness = {node: closeness for (node, closeness) in nx.closeness_centrality(G).items()}
    betweenness = {node: betweenness for (node, betweenness) in nx.betweenness_centrality(G).items()}
    pagerank = {node: pagerank for (node, pagerank) in nx.pagerank(G).items()}

    nodes = []
    # graph_layout[dct["id"]] = {"x": dct["x"], "y": dct["y"]}
    for n, data in list(G.nodes(data=True)):
        dct = {
            "id": n, "degree": degree[n], "closeness": closeness[n], 
            "betweenness": betweenness[n], "page_rank": pagerank[n], "display": "True"
        }
        if graph_layout != {}:
            dct["x"] = graph_layout[n]["x"]
            dct["y"] = graph_layout[n]["y"]
        nodes.append(dct)

    # add a pesudo node as center node
    if len(list(nx.connected_components(G))) > 1:
        nodes.append({
            "id": "source", "degree": -1, "closeness": -1, "betweenness": -1, "page_rank": -1, "display": "False"
        })

    return nodes

def G_links(G):
    links = []
    for (i, j) in G.edges():
        links.append({"source": i, "target": j, 'dashed': "False", "display": "True"})
    
    if len(list(nx.connected_components(G))) > 1:
        for CC in nx.connected_components(G):
            subgraph = G.copy().subgraph(CC)
            # find highest degree node 
            keys = list(nx.degree_centrality(subgraph).keys())
            values = list(nx.degree_centrality(subgraph).values())
            node = keys[ np.argmax(values)]
            links.append({"source": "source", "target": node, 'dashed': "False", "display": "False"})
    return links

def read_sample(sample):
    assert sample in [
        "911", "everett", "borgatti", "potts", "DOMESTICTERRORWEB", 
        "HEROIN_DEALING", "MAIL",  "suicide", "SWINGERS_club", "HAMBURG_TIE_YEAR"
    ]

    if sample == "911":
        G = nx.read_gml("./911.gml")
    elif sample in ["everett", "potts", "borgatti"]:
        G = nx.read_gml(f"./sample_data/{sample}.gml")
    else:
        G = nx.read_gml(f"./empirical_data/{sample}.gml")

    map_dct = {node: int(idx) for idx, node in enumerate(G.nodes())}
    return nx.relabel_nodes(G, map_dct, copy=True)
    


In [21]:
def node_centrality_criteria(G):

    centrality = {
        "degree": {"node":[], "value":[]}, "closeness": {"node":[], "value":[]}, "betweenness": {"node":[], "value":[]}, "page_rank": {"node":[], "value":[]}
    }
    for metric in ["degree", "closeness", "betweenness", "page_rank"]:
        node_lst = []
        for node in G_nodes(G):
            node_lst.append((node["id"], node[metric]))
        
        hxa = sorted(node_lst, key=lambda x: x[1], reverse=True)
        rank = 1
        now_score = hxa[0][1]
        for idx, (node, score) in enumerate(hxa): 
            if now_score > score:
                now_score = score
                rank += 1
            centrality[metric]["node"].append(node)
            centrality[metric]["value"].append(rank)

    return centrality